In [1]:
%matplotlib inline
#from query.datasets.prelude import *
import numpy
import pysrt
import textacy
import gensim

In [2]:
# Load transcripts and compute time offsets for captions
transcripts = []
#for video in Video.objects.exclude(path__contains='segment'):
#    base = os.path.split(os.path.splitext(video.path)[0])[1]
     #subs = pysrt.open('/app/subs/{}.cc5.srt'.format(base))
subs = pysrt.open('../data/videos/CNNW_20120201_160000_CNN_Newsroom.cc5.srt')
offsets = []
transcript = ''
for sub in subs:
    offsets.append((len(transcript), sub.start, sub.end))
    transcript += sub.text.replace('\n', ' ').replace('>', ' ') + ' '
transcripts.append((None, transcript, offsets))

In [19]:
subs[0].start
subs[0].end
subs[0].text

In [3]:
# Convert transcripts into textacy Docs (implicitly performing various NLP tasks)
#docs = [textacy.doc.Doc(transcript, lang=u'en', metadata={'path': video.path, 'offsets': offsets}) for (video, transcript, offsets) in transcripts]
docs = [textacy.doc.Doc(transcript, lang=u'en', metadata={'offsets': offsets}) for (video, transcript, offsets) in transcripts]
corpus = textacy.Corpus(u'en', docs=docs)

In [4]:
# Helper functions
def token_time(doc, tok):
    tok_offset = tok.idx
    for i, (offset, start, end) in enumerate(doc.metadata['offsets']):
        if tok_offset < offset:
            return (start, end)
    raise Exception("No token time?")
    
def compute_tfidf(corpus):
    gensim_dict = gensim.corpora.Dictionary(corpus)
    gensim_corpus = [gensim_dict.doc2bow(text) for text in transcript_words]
    tfidf_model = gensim.models.tfidfmodel.TfidfModel(gensim_corpus, normalize=True)
    return {gensim_dict.get(id): value for doc in tfidf[gensim_corpus] for id, value in doc}

In [6]:
# Load Word2Vec model
model = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True)

def sim_score(a, b):
    try:
        return model.wv.similarity(a, b)
    except KeyError:
        return 0

In [7]:
model.wv.similarity('economy', 'jobs')

0.40456532973171394

In [8]:
# TODO: incorporate TFIDF segments

topics = ['immigration', 'china', 'syria', 'terrorism', 'economy', 'election']
transcript_tokens = [
    list(textacy.extract.words(doc, filter_nums=True, include_pos=['PROPN', 'NOUN'])) 
    for doc in corpus.docs]
  
N = len(topics)    
#fig, ax = plt.subplots(2, N / 2, figsize=(18, 16))
#ax = ax.flatten()
W = 30
MENTION_THRESHOLD = 3
PEAK_DURATION_THRESHOLD = 5

def topic_peaks(topic, tokens):
    scores = np.array([sim_score(token.text.lower(), topic) for token in tokens])
    scores = scores > 0.6
    sums = np.array([np.sum(scores[j:j+W]) for j in range(W, len(scores) - W)]) >= MENTION_THRESHOLD
    start = None
    peaks = []
    for j in range(len(sums)):
        if sums[j] and start is None:
            start = j
        elif not sums[j] and start is not None:
            idx = (start + j) / 2 + W
            peak_duration = j - start
            if peak_duration > PEAK_DURATION_THRESHOLD:
                peaks.append((tokens[idx], peak_duration))
            start = None

    #ax[i].set_ylim([0, 1])
    #ax[i].set_title(topic)
    #ax[i].plot(sums)
    
    return peaks


topic_labeler, _ = Labeler.objects.get_or_create(name='word2vec')
topic_tracks = []
for tokens, doc in zip(transcript_tokens, corpus.docs):
    print(doc.metadata['path'])
    video = Video.objects.get(path=doc.metadata['path'])
    for i, topic in enumerate(topics): 
        peaks = topic_peaks(topic, tokens)
        print(topic, peaks)
        for (token, _) in peaks:
            (start, end) = token_time(doc, token)
            start = start.to_time()
            seconds = (((start.hour * 60) + start.minute) * 60 + start.second)
            
            # Naively assume 4 minute segment size
            min_time = max(seconds - 2 * 60, 0)
            max_time = min(seconds + 2 * 60, int(video.num_frames / video.fps))
            
            topic_model, _ = Topic.objects.get_or_create(name=topic)
            track = TopicTrack(
                video=video, min_frame=min_time*video.fps, max_frame=max_time*video.fps, topic=topic_model,
                labeler=topic_labeler)
            topic_tracks.append(track)
#_ = TopicTrack.objects.bulk_create(topic_tracks)            

NameError: name 'Labeler' is not defined